In [1]:
import sys
#edit below to include your directory as needed
#sys.path.append("/Users/blatham/Hacktoberfest21")
#sys.path.append("/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages")
import gmsh
import numpy as np

In [2]:
#begin Model
gmsh.initialize()
gmsh.model.add('clover_mesh')

In [3]:
#Parameters
h = 0.1
l = 0
L = 2
delta = 0.5
LL = L+ delta
Angle1 = np.pi/4
Angle2 = np.pi/6
C = np.cos(Angle1)
S = np.sin(Angle1)
Cp = np.cos(Angle1 + Angle2)
Cm = np.cos(Angle1 - Angle2)
Sp = np.sin(Angle1 + Angle2)
Sm = np.sin(Angle1 - Angle2)
Cph = np.cos(Angle1 + 0.5 * Angle2)
Cmh = np.cos(Angle1 - 0.5 * Angle2)
Sph = np.sin(Angle1 + 0.5 * Angle2)
Smh = np.sin(Angle1 - 0.5 * Angle2)
Rx = np.abs(L*Cp - 0.5*L*(Cp+0.5*(Cp+Cm)))
Ry = np.abs(L*Sp - 0.5*L*(Sp+0.5*(Sp+Sm)))
R = np.sqrt(Rx**2 + Ry**2)
InnerCircleRadius=0.3

In [4]:
#Points
Points=[]
Points.append((l*Cp, l*Sp, 0, h))
Points.append((L*Cp, L*Sp, 0, h))
Points.append((0.5* L* (Cp+Cm), 0.5* L* (Sp+Sm), 0, h))
#The below point is commented out in the original mesh - not sure if it is to be included.
Points.append((l*Cm, l*Sm, 0, h))
Points.append((L*Cm, L*Sm, 0, h))
Points.append((0.5*L*(Cp+0.5*(Cp+Cm)), 0.5*L*(Sp+0.5*(Sp+Sm)), 0, h))
Points.append((0.5*L*(Cm+0.5*(Cp+Cm)), 0.5*L*(Sm+0.5*(Sp+Sm)), 0, h))
Points.append((0.5*L*(Cp+0.5*(Cp+Cm))+R*np.cos(np.pi/4), 0.5*L*(Sp+0.5*(Sp+Sm))+R*np.sin(np.pi/4), 0, h))
Points.append((0.5*L*(Cm+0.5*(Cp+Cm))+R*np.cos(np.pi/4), 0.5*L*(Sm+0.5*(Sp+Sm))+R*np.sin(np.pi/4), 0, h))

Points.append((InnerCircleRadius*Cp,InnerCircleRadius*Sp,0,h))
Points.append((InnerCircleRadius*Cm,InnerCircleRadius*Sm,0,h))
Points.append((InnerCircleRadius,0,0,h))
Points.append((0,InnerCircleRadius,0,h))
#Points.append((InnerCircleRadius*np.cos(np.pi/4),InnerCircleRadius*(np.cos(np.pi/4)),0,h))
geoPoints= [gmsh.model.geo.addPoint(*p) for p in Points]

In [5]:
geoPoints

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [6]:
#Lines
Lines=[]
Lines.append((10,2))
Lines.append((11,5))
#Lines.append((1,10))
#Lines.append((1,11))
Lines.append((1,12))
Lines.append((1,13))
geoLines=[gmsh.model.geo.addLine(*L) for L in Lines]

In [7]:
#Circles
Circles=[]
Circles.append((2,6,8))
Circles.append((8,6,3))
Circles.append((3,7,9))
Circles.append((9,7,5))
Circles.append((12,1,11))
Circles.append((10,1,13))

geoCircles=[gmsh.model.geo.addCircleArc(*C) for C in Circles]

In [8]:
#loop and surface
gmsh.model.geo.addCurveLoops((3,9,1,5,6,7,8,-2,10,-4))

array([1], dtype=int32)

In [9]:
Surfaces=[]
gmsh.model.geo.addPlaneSurface([1])

1

In [10]:
Surfaces.append([(2,1)])
Surfaces.append(gmsh.model.geo.copy([(2,1)]))
gmsh.model.geo.symmetrize(Surfaces[1],1,0,0,0)
Surfaces.append(gmsh.model.geo.copy([(2,1)]))
gmsh.model.geo.symmetrize(Surfaces[2],0,1,0,0)
Surfaces.append(gmsh.model.geo.copy([(2,1)]))
gmsh.model.geo.symmetrize(Surfaces[3],1,1,0,0)

In [11]:
gmsh.model.geo.addSurfaceLoop((1,11,21,32))

1

In [12]:
Interface=gmsh.model.geo.addPhysicalGroup(1,(9,2,8,7,6,5,-1,10,20,19,18,17,16,15,-14,13,41,40,39,38,37,36,-35,34,30,29,28,27,26,25,-24,23))
gmsh.model.setPhysicalName(1,Interface,'Interface')

In [13]:
clover=gmsh.model.geo.addPhysicalGroup(2,(1,11,21,32))
gmsh.model.setPhysicalName(2,clover,'clover')

In [14]:
gmsh.model.geo.synchronize()

In [15]:
gmsh.model.getEntities()

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (0, 11),
 (0, 12),
 (0, 13),
 (0, 15),
 (0, 20),
 (0, 24),
 (0, 28),
 (0, 29),
 (0, 34),
 (0, 38),
 (0, 39),
 (0, 44),
 (0, 48),
 (0, 55),
 (0, 59),
 (0, 63),
 (0, 64),
 (0, 69),
 (0, 73),
 (0, 74),
 (0, 79),
 (0, 83),
 (0, 88),
 (0, 95),
 (0, 99),
 (0, 103),
 (0, 104),
 (0, 109),
 (0, 113),
 (0, 114),
 (0, 119),
 (0, 123),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 10),
 (1, 12),
 (1, 13),
 (1, 14),
 (1, 15),
 (1, 16),
 (1, 17),
 (1, 18),
 (1, 19),
 (1, 20),
 (1, 23),
 (1, 24),
 (1, 25),
 (1, 26),
 (1, 27),
 (1, 28),
 (1, 29),
 (1, 30),
 (1, 31),
 (1, 34),
 (1, 35),
 (1, 36),
 (1, 37),
 (1, 38),
 (1, 39),
 (1, 40),
 (1, 41),
 (2, 1),
 (2, 11),
 (2, 21),
 (2, 32)]

In [16]:
#meshing
gmsh.model.mesh.generate(2)
gmsh.option.setNumber("Mesh.MshFileVersion",2.2)

gmsh.write('clover_mesh.msh')

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 20%] Meshing curve 5 (Circle)
Info    : [ 20%] Meshing curve 6 (Circle)
Info    : [ 20%] Meshing curve 7 (Circle)
Info    : [ 20%] Meshing curve 8 (Circle)
Info    : [ 30%] Meshing curve 9 (Circle)
Info    : [ 30%] Meshing curve 10 (Circle)
Info    : [ 30%] Meshing curve 12 (Line)
Info    : [ 40%] Meshing curve 13 (Circle)
Info    : [ 40%] Meshing curve 14 (Line)
Info    : [ 40%] Meshing curve 15 (Circle)
Info    : [ 40%] Meshing curve 16 (Circle)
Info    : [ 50%] Meshing curve 17 (Circle)
Info    : [ 50%] Meshing curve 18 (Circle)
Info    : [ 50%] Meshing curve 19 (Line)
Info    : [ 50%] Meshing curve 20 (Circle)
Info    : [ 60%] Meshing curve 23 (Circle)
Info    : [ 60%] Meshing curve 24 (Line)
Info    : [ 60%] Meshing curve 25 (Circle)
Info    : [ 70%] Meshing curve 26 (Circle)
Info    : [ 

In [17]:
#optional gui cell to see mesh, uncomment if wanted
#gmsh.fltk.run()

In [18]:
#closing

gmsh.finalize()